# 1 Microsoft OmniParser

https://github.com/microsoft/OmniParser \
https://microsoft.github.io/OmniParser/ \
https://huggingface.co/microsoft/OmniParser \
\
OmniParser was the first model that I tested. My impressions after playing with its demo were grand (you can find it here: https://huggingface.co/spaces/microsoft/OmniParser); after providing it with a picture of Lidl's promo magazine, it was able to extract text from it with very high precision. Additionally, because it wasn't parsing any XML trees, output was very clear; as a result I got a list of extracted strings and a list of their bounding boxes. Using clustering or even simple algorithms to bind together close elements could provide us with a very clean data that later BERT model could easily label. \
Problems arise with the deployment of the model. As of today (12.11.24) I couldn't find any official data regarding the number of params of the model. Additionally, its deployment on a local machine is non-trivial: it's huggingface page doesn't hold all necessary files, so to run the model, you have to clone the github repo AND THEN download files from the huggingface page. And when I finally managed to do that, I found that this model can't be run without NVIDIA GPU drivers (I'm using an AMD GPU). \
In conclusion, I think that OmniParser could prove very useful to us, but we won't be able to deploy it to the mobile.

# 2 MoondreamAI

https://huggingface.co/vikhyatk/moondream2 \
https://github.com/vikhyat/moondream \
https://moondream.ai/ \


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image

model_id = "vikhyatk/moondream2"
revision = "2024-08-26"
model = AutoModelForCausalLM.from_pretrained(
    model_id, trust_remote_code=True, revision=revision
)
tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)

image = Image.open('Screenshot from 2024-11-10 18-13-40.png')
enc_image = model.encode_image(image)
print(model.answer_question(enc_image, "Describe this image.", tokenizer))

PhiForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


The image is a promotional flyer for a toy store featuring two product images: a child playing with a wooden toy set and a magnetic toy set.


Above you can see a demo representing the MoonDreamAI2. As you can see, instead of operating on a raw data, we can prompt this model for a description of an image. Model itself has around 1.8B params and is marketed as something that should be able to run on edge devices (such as mobile phones). Additionally, it's supposed to be easily configurable to be smaller (on its official page it is said that it can be easily distilled). Sadly, I didn't find any examples of that other than HF tutotial on distilling Vision models (can be found here: https://github.com/johko/computer-vision-course/blob/main/notebooks/Unit%203%20-%20Vision%20Transformers/KnowledgeDistillation.ipynb). Still, there is an option for batch requests, as well as a flash attention, which should speed-up the model (presented below). \

In [ ]:
model_id = "vikhyatk/moondream2"
revision = "2024-08-26"
model = AutoModelForCausalLM.from_pretrained(
    model_id, trust_remote_code=True, revision=revision
)
tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision,
                                          attn_implementation="flash_attention_2") # flash_attention

image = Image.open('Screenshot from 2024-11-10 18-13-40.png')
enc_image = model.encode_image(image)
print(model.answer_question(enc_image, "Describe this image.", tokenizer))

If your laptop/IDE crashed during the execution of any of those two code snippetss, don't you worry; at least on my machine, one run of this model is enough for my laptop. Which contradicts with the statement that this model is supposed to be run on edge devices. \
Another issue is that this model is, at the end of the day, too dumb for our needs. You can play with it safely on its demo page, and you will probably quickly notice that while it responds very well to prompts like "Describe me this", modifying the prompt to something like "Describe me this AS json" will cause the model to generate random sequences of words from the picture.
In conclusion, I'm not sure whether this would run on a mobile device (at least without distillation), and even if, it would be nice for tasks related to image description, but we need something much more specific.